Архитектуры нейронных сетей

Импорт модулей

In [2]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Архитектура нейронных сетей

In [3]:
class EmbeddingAudio(nn.Module):
  def __init__(self, shape):
    super(EmbeddingAudio, self).__init__()
    x, y = shape[0], (((shape[1]-4)//2-4)//2-4)//2-4
    self.conv = nn.Sequential(nn.Conv2d(1, 27, (1,3)), nn.BatchNorm2d(27), nn.ReLU(),
                              nn.Conv2d(27, 27, (1,3)), nn.BatchNorm2d(27), nn.ReLU(),
                              nn.MaxPool2d((1, 2)),
                              nn.Conv2d(27, 81, (1,3)), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.Conv2d(81, 81, (1,3)), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.MaxPool2d((1, 2)),
                              nn.Conv2d(81, 81, (1,3)), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.Conv2d(81, 81, (1,3)), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.MaxPool2d((1, 2)),
                              nn.Conv2d(81, 81, (1,3)), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.Conv2d(81, 81, (1,3)), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.Flatten())
    self.fc = nn.Sequential(nn.Linear(x*y*81, 2000), nn.BatchNorm1d(2000), nn.ReLU(),
                            nn.Linear(2000, 500), nn.BatchNorm1d(500), nn.ReLU(),)
  def forward(self, x):
    z1 = self.conv(x)
    z2 = self.fc(x)
    return z2

In [8]:
class EmbeddingImage(nn.Module):
  def __init__(self, shape):
    super(EmbeddingImage, self).__init__()
    x, y = (((shape[0]-4)//2-4)//2-4)//2-4, (((shape[1]-4)//2-4)//2-4)//2-4
    self.conv = nn.Sequential(nn.Conv2d(3, 27, 3), nn.BatchNorm2d(27), nn.ReLU(),
                              nn.Conv2d(27, 27, 3), nn.BatchNorm2d(27), nn.ReLU(),
                              nn.MaxPool2d(2),
                              nn.Conv2d(27, 81, 3), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.Conv2d(81, 81, 3), nn.BatchNorm2d(81), nn.ReLU(),
                              nn.MaxPool2d(2),
                              nn.Conv2d(81, 243, 3), nn.BatchNorm2d(243), nn.ReLU(),
                              nn.Conv2d(243, 243, 3), nn.BatchNorm2d(243), nn.ReLU(),
                              nn.MaxPool2d(2),
                              nn.Conv2d(243, 243, 3), nn.BatchNorm2d(243), nn.ReLU(),
                              nn.Conv2d(243, 243, 3), nn.BatchNorm2d(243), nn.ReLU(),
                              nn.Flatten())
    self.fc = nn.Sequential(nn.Linear(x*y*243, 2000), nn.BatchNorm1d(2000), nn.ReLU(),
                            nn.Linear(2000, 500), nn.BatchNorm1d(500), nn.ReLU(),)
  def forward(self, x):
    z1 = self.conv(x)
    z2 = self.fc(x)
    return z2

In [5]:
class LSTM(nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()
    self.lstm = nn.LSTM(1000, 300)
    self.fc = nn.Linear(300, 1)
  def forward(self, x):
    h0 = torch.zeros(1, x.shape[0], 300).to(dev)
    c0 = torch.zeros(1, x.shape[0], 300).to(dev)
    out, (hn, cn) = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])
    return out, hn, cn

Создание объектов классов

In [6]:
a = EmbeddingAudio((128, 400))

In [9]:
b = EmbeddingImage((300, 300))

In [10]:
c = LSTM()